# Tidy

In [28]:
import pandas as pd
import numpy as np
import copy 

#Read data from csv file
ds = pd.read_csv('../OriginalData/tb.csv')

#Show 5 first records
ds.head(5)

,iso2,year,new_sp,new_sp_m04,new_sp_m514,new_sp_m014,new_sp_m1524,new_sp_m2534,new_sp_m3544,new_sp_m4554,...,new_sp_f04,new_sp_f514,new_sp_f014,new_sp_f1524,new_sp_f2534,new_sp_f3544,new_sp_f4554,new_sp_f5564,new_sp_f65,new_sp_fu
0,AD,1989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AD,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AD,1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AD,1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AD,1993,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
pd.options.mode.chained_assignment = None  # default='warn'

#Copying dataset 
ds_clear = copy.deepcopy(ds)

#Changing name 'iso2' to 'city' which is easier to understand
ds_clear.rename(columns={'iso2': 'city'}, inplace=True)

#Melting around 'city' and 'year' to make our table taller 
ds_clear = ds_clear.melt(id_vars=['city', 'year'], var_name='temp', value_name='count')

#Deleting rows where count (number of ill person) is NaN
ds_clear = ds_clear[ds_clear['count'] != 'NaN']

#Deleting rows with count = 'NaN'
ds_clear.dropna(axis='rows', inplace=True)

#Deleting rows where we do not know sex
ds_clear = ds_clear[ds_clear.temp != 'new_sp']

#Deleting unnecesary part of strings
ds_clear['temp'] = ds_clear.temp.str.slice(start=7)

#Preparing column named 'sex' which contains info about gender
ds_clear['sex'] = ds_clear.temp.str.slice(start=0, stop=1)

#Making names of genders explicit
ds_clear.sex[ds_clear['sex'] == 'm'] = 'male'
ds_clear.sex[ds_clear['sex'] == 'f'] = 'female'

#Preparing column name temp to present age range
ds_clear['temp'] = ds_clear.temp.str.slice(start=1)

#Function which makes ranges prettier
def change_range(x):
    result =''
    if x == '65':
        result = x + '+'
    if len(x) in [2, 3] and x != '65':
        result = x[:1] + '-' + x[1:]
    if len(x) == 4:
         result = x[:2] + '-' + x[2:]

    return result

#Changing displaying of age ranges 
ds_clear['temp'] = ds_clear['temp'].apply(change_range)

C:\Users\kacpe\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [30]:
print('Information about sex: ')
print(ds_clear.groupby('sex').count())

Information about sex: 
         city   year   temp  count
sex                               
female  17731  17731  17731  17731
male    17821  17821  17821  17821


In [31]:
print('Information about age ranges: ')
print(ds_clear.groupby('temp').count())

Information about age ranges: 
       city  year  count   sex
temp                          
        543   543    543   543
0-14   4731  4731   4731  4731
0-4     785   785    785   785
15-24  4773  4773   4773  4773
25-34  4780  4780   4780  4780
35-44  4785  4785   4785  4785
45-54  4794  4794   4794  4794
5-14    805   805    805   805
55-64  4780  4780   4780  4780
65+    4776  4776   4776  4776


In [32]:
#Rename column temp
ds_clear.rename(columns={'temp': 'age'}, inplace = True)

print('Results of cleaning data: ')
ds_clear.head(10)

Results of cleaning data: 


,city,year,age,count,sex
5784,AD,2005,0-4,0.0,male
5785,AD,2006,0-4,0.0,male
5787,AD,2008,0-4,0.0,male
5811,AE,2006,0-4,0.0,male
5812,AE,2007,0-4,0.0,male
5813,AE,2008,0-4,0.0,male
5863,AG,2007,0-4,0.0,male
5910,AL,2005,0-4,0.0,male
5911,AL,2006,0-4,1.0,male
5912,AL,2007,0-4,0.0,male


In [33]:
#Saving results to new csv file 
ds_clear.to_csv(path_or_buf='../AnalysisData/tb_clean.csv', index=False)